<a href="https://colab.research.google.com/github/JI411/min_prirodi_soyuz/blob/main/princess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скрипт для второго задания

## Загружаем данные и модель, делаем импорты

In [1]:
!nvidia-smi

Thu Dec  2 16:22:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install gdown
!gdown --id 1BIdGch53zM5styGP9KFBvUS53yTNgIZ8  # https://drive.google.com/file/d/1BIdGch53zM5styGP9KFBvUS53yTNgIZ8/view
!gdown --id 1JJyEalh0oPLowHkQIykCK2ctETEOEzjG  # https://drive.google.com/file/d/1JJyEalh0oPLowHkQIykCK2ctETEOEzjG/view
!unzip sample.zip

In [3]:
import torch
import argparse
from pathlib import Path
from typing import List
import pandas as pd
from tqdm import tqdm

## Предсказываем

In [4]:
def get_batches(images: List[Path], batch_size: int):
    """
    Генерирует батч из путей в images
    :param images: список путей к изображениям
    :param batch_size: сколько путей возвращать за итерацию 
    """
    for i in range(0, len(images), batch_size):
        yield images[i: i + batch_size]

In [5]:
def download_model(weights_path: Path):
    """
    Загружает модель и настраевает параметры
    :param weights_path: локальный путь к весам
    :return: pytorch модель
    """
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    model.conf = 0.25  # NMS confidence threshold
    model.iou = 0.45  # NMS IoU threshold
    model.multi_label = True  # NMS multiple labels per box
    model.max_det = 10  # maximum number of detections per image
    return model

In [6]:
def predict(images_dir: Path, weights_path: Path, batch_size: int = 20) -> None:
    """
    Инференс по батчам картинок. Отделяет Принцессу от других тигров.
    Сохраняет фото с bbox и csv-таблицу с результатами для подсчёта представленной организаторами метрики.

    :param images_dir: папка с изображениями
    :param weights_path: путь до файла с весами
    :param batch_size: сколько изображений за раз обрабатывать
    """
    model = download_model(weights_path)
    images = list(images_dir.glob('*.jpg'))
    print('Всего изображений:', len(images))
    result = []
    for batch in tqdm(get_batches(images=images, batch_size=batch_size)):
        img_paths = batch[:]
        output = model(batch, size=640)
        output.save('/content/output/')
        for img_path, img_output in zip(img_paths, output.xyxy):
            try:
                label = img_output.tolist()[0][-1]
            except IndexError:
                label = 0
            result.append({'id': img_path.name, 'class': label})
    result = pd.DataFrame(result)
    result.to_csv('labels.csv', index=False)

In [7]:
dir = '/content/sample/'
weights = '/content/_princess30.pt'
predict(images_dir=Path(dir), weights_path=Path(weights))

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-12-2 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 574 layers, 139980484 parameters, 0 gradients
Adding AutoShape... 


Всего изображений: 9


0it [00:00, ?it/s]Saved 9 images to /content/output
1it [00:02,  2.79s/it]
